## the aim of this work is to take the cpio anotation data of this work https://github.com/OpenDriveLab/OpenLane/tree/main/data and convert it to coco format

In [1]:
import json
import os
from PIL import Image


def list_sub_directories(dir_name):
    list_of_dir = []
    with os.scandir(dir_name) as entries:
        for entry in entries:
            if entry.is_dir():
                x = entry.name
                x = dir_name + '/' + x
                list_of_dir.append(x)
    return list_of_dir


def cpio_to_coco(parent_dir, training_or_validation):
    
    coco_data = {
        "info": {},
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": [] 
    }
    coco_data["categories"].append({"id": 1, "name": "level-1", "supercategory": "mobile-1"})
    coco_data["categories"].append({"id": 2, "name": "level-2", "supercategory": "mobile-2"})
    coco_data["categories"].append({"id": 3, "name": "level-3", "supercategory": "mobile-3"})
    coco_data["categories"].append({"id": 4, "name": "level-4", "supercategory": "mobile-4"})

    image_id_counter = 0
    annotation_id_counter = 0
    
    list_of_dir = list_sub_directories(parent_dir)
    
    for annotations_dir in list_of_dir:
        for filename in os.listdir(annotations_dir):
            if filename.endswith(".json"):
                image_filename = filename.replace(".json", "")
                with open(os.path.join(annotations_dir, filename), 'r') as f:
                    image_annotations = json.load(f)
                image_relative_path = image_annotations["raw_file_path"]
                image_relative_path = image_relative_path.split(training_or_validation+'/')  #will be used in cocodata
                 
                img = Image.open("data_for_cpio/images/images/"+image_annotations["raw_file_path"])
                width, height = img.size
                image_id = image_id_counter
                coco_data["images"].append({
                    "id": image_id,
                    "width": width, # Assuming this is in your individual file
                    "height": height, # Assuming this is in your individual file
                    "file_name": image_relative_path[1]
                })
                image_id_counter += 1
                
                if len(image_annotations["result"])>0:
                    for obj in image_annotations["result"]: # Assuming 'objects' list in your individual file
                        if 'id' in obj:
                            category_id = obj["id"]
                            bb_x = obj["x"]
                            bb_y = obj["y"]
                            bb_width = obj["width"]
                            bb_height = obj["height"]
                            coco_data["annotations"].append({
                                "id": annotation_id_counter,
                                "image_id": image_id,
                                "category_id": category_id,
                                "bbox": [bb_x, bb_y, bb_width, bb_height],
                                "area": bb_width * bb_height,
                                "iscrowd": 0,
                                "segmentation": 0
                            })
                            annotation_id_counter += 1  
                        else:
                            pass
                else:
                    pass

    with open(training_or_validation +'_all'+'.json', 'w') as f:
        json.dump(coco_data, f, indent=4)
        


if __name__ == "__main__": 
    cpio_to_coco("data_for_cpio/cipo/cipo/training", "training")
    cpio_to_coco("data_for_cpio/cipo/cipo/validation", "validation")